In [10]:
#!pip install cryptography

## Overview

This notebook contains a collection of examples of how to work with OpenSSL in Python. My purpose is to give a complete example of how to generate a Certificate Authority, create a Certificate Recovation List, create certificates and keys for clients and how to revoke these certificates.

openssl ca -gencrl -out ca.crl -cert ca.crt -keyfile ca.key

### Imports

In [61]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import (Encoding, PrivateFormat, NoEncryption)

from cryptography.hazmat.primitives import serialization
import uuid
import datetime

### 1. Generate Root CA

encyption password: test

#### OpenSSL command

openssl genrsa -des3 -out ca.key 2048

openssl req -new -x509 -outform PEM -days 3650 -key ca.key -out ca.crt

In [87]:

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()
builder = x509.CertificateBuilder()
builder = builder.subject_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u'Test Root CA')
]))
builder = builder.issuer_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u'Test Root CA'),
]))
builder = builder.not_valid_before(datetime.datetime.now()-datetime.timedelta(hours=1))
builder = builder.not_valid_after(datetime.datetime.now()+datetime.timedelta(7))
builder = builder.serial_number(int(uuid.uuid4()))
builder = builder.public_key(public_key)
builder = builder.add_extension(
    x509.BasicConstraints(ca=True, path_length=None), critical=True,
)
certificate = builder.sign(
    private_key=private_key, algorithm=hashes.SHA256(),
    backend=default_backend()
)

#print(isinstance(certificate, x509.Certificate))

with open("openssl/ca.key", "wb") as f:
    f.write(private_key.private_bytes(Encoding.PEM, PrivateFormat.TraditionalOpenSSL,\
                                      encryption_algorithm=serialization.BestAvailableEncryption(b"test")))

with open("openssl/ca.crt", "wb") as f:
    f.write(certificate.public_bytes(
        encoding=serialization.Encoding.PEM,
    ))
    
print(certificate.not_valid_before)
print(certificate.not_valid_after)

2020-11-10 10:52:27
2020-11-17 11:52:27


### 2. Generate empty CRL

#### OpenSSL command

openssl ca -gencrl -out ca.crl -cert ca.crt -keyfile ca.key

In [96]:

pem_cert = open("openssl/ca.crt","rb").read()
ca_crt = x509.load_pem_x509_certificate(pem_cert, default_backend())
pem_key = open("openssl/ca.key","rb").read()
ca_key = serialization.load_pem_private_key(pem_key, password=b"test", backend=default_backend())


builder = x509.CertificateRevocationListBuilder()
#builder = builder.add_extension(x509.CRLNumber(1000), critical=False)
builder = builder.last_update(datetime.datetime.now()-datetime.timedelta(hours=1))
builder = builder.next_update(datetime.datetime.now() + datetime.timedelta(1) - datetime.timedelta(hours=1))
builder = builder.issuer_name(ca_crt.issuer)

cert_revocation_list = builder.sign(private_key=ca_key,algorithm=hashes.SHA256(),backend=default_backend())

with open("openssl/ca.crl","wb") as f:
    f.write(cert_revocation_list.public_bytes(serialization.Encoding.PEM))

### 3. Server

#### OpenSSL command
openssl genrsa -out server.key 2048

openssl req -new -out server.csr -key server.key

openssl x509 -req -in server.csr -CA ca.crt -CAkey ca.key -CAcreateserial -outform PEM -out server.crt -days 720

In [89]:

private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())

builder = x509.CertificateSigningRequestBuilder()
builder = builder.subject_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u"server")
]))

builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True,)

request = builder.sign(private_key, hashes.SHA256(), default_backend())

with open("openssl/server.csr","wb") as f:
    f.write(request.public_bytes(Encoding.PEM))

with open("openssl/server.key","wb") as f:
    f.write(private_key.private_bytes(Encoding.PEM, PrivateFormat.TraditionalOpenSSL, NoEncryption()))


pem_csr = open("openssl/server.csr","rb").read()
csr = x509.load_pem_x509_csr(pem_csr, default_backend())

pem_cert = open("openssl/ca.crt","rb").read()
ca = x509.load_pem_x509_certificate(pem_cert, default_backend())
pem_key = open("openssl/ca.key","rb").read()
ca_key = serialization.load_pem_private_key(pem_key, password=b"test", backend=default_backend())

#print(csr.subject)

builder = x509.CertificateBuilder()
builder = builder.subject_name(csr.subject)
builder = builder.issuer_name(ca.subject)

print(csr.subject)

#builder = builder.not_valid_before(datetime.datetime.now())
builder = builder.not_valid_before(datetime.datetime.now()-datetime.timedelta(hours=1))
builder = builder.not_valid_after(datetime.datetime.now()+datetime.timedelta(7)) # days
builder = builder.public_key(csr.public_key())
builder = builder.serial_number(int(uuid.uuid4()))

for ext in csr.extensions:
    builder = builder.add_extension(ext.value, ext.critical)
    
certificate = builder.sign(private_key=ca_key, algorithm=hashes.SHA256(),backend=default_backend())

with open("openssl/server.crt","wb") as f:
    f.write(certificate.public_bytes(serialization.Encoding.PEM))

<Name(CN=server)>
<Name(CN=Test Root CA)>


### 4. Client 1

#### OpenSSL command
openssl genrsa -out client.key 2048

openssl req -new -out client.csr -key client.key

openssl x509 -req -in client.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out client.crt -days 720

In [90]:
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())

builder = x509.CertificateSigningRequestBuilder()
builder = builder.subject_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u"client 1")
]))

builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True,)

request = builder.sign(private_key, hashes.SHA256(), default_backend())

with open("openssl/client1.csr","wb") as f:
    f.write(request.public_bytes(Encoding.PEM))

with open("openssl/client1.key","wb") as f:
    f.write(private_key.private_bytes(Encoding.PEM, PrivateFormat.TraditionalOpenSSL, NoEncryption()))

pem_csr = open("openssl/client1.csr","rb").read()
csr = x509.load_pem_x509_csr(pem_csr, default_backend())

pem_cert = open("openssl/ca.crt","rb").read()
ca = x509.load_pem_x509_certificate(pem_cert, default_backend())
pem_key = open("openssl/ca.key","rb").read()
ca_key = serialization.load_pem_private_key(pem_key, password=b"test", backend=default_backend())

print(csr.subject)

builder = x509.CertificateBuilder()
builder = builder.subject_name(csr.subject)
builder = builder.issuer_name(ca.subject)
#builder = builder.not_valid_before(datetime.datetime.now())
builder = builder.not_valid_before(datetime.datetime.now()-datetime.timedelta(hours=1))
builder = builder.not_valid_after(datetime.datetime.now()+datetime.timedelta(7)) # days
builder = builder.public_key(csr.public_key())
builder = builder.serial_number(int(uuid.uuid4()))

for ext in csr.extensions:
    builder = builder.add_extension(ext.value, ext.critical)
    
certificate = builder.sign(private_key=ca_key, algorithm=hashes.SHA256(),backend=default_backend())

print(certificate.serial_number)

with open("openssl/client1.crt","wb") as f:
    f.write(certificate.public_bytes(serialization.Encoding.PEM))

<Name(CN=client 1)>
183256968768535528365450232687811955506


### 5. Client 2

In [91]:
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())

builder = x509.CertificateSigningRequestBuilder()
builder = builder.subject_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u"client 2")
]))

builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True,)

request = builder.sign(private_key, hashes.SHA256(), default_backend())

with open("openssl/client2.csr","wb") as f:
    f.write(request.public_bytes(Encoding.PEM))

with open("openssl/client2.key","wb") as f:
    f.write(private_key.private_bytes(Encoding.PEM, PrivateFormat.TraditionalOpenSSL, NoEncryption()))
    
pem_csr = open("openssl/client2.csr","rb").read()
csr = x509.load_pem_x509_csr(pem_csr, default_backend())

pem_cert = open("openssl/ca.crt","rb").read()
ca = x509.load_pem_x509_certificate(pem_cert, default_backend())
pem_key = open("openssl/ca.key","rb").read()
ca_key = serialization.load_pem_private_key(pem_key, password=b"test", backend=default_backend())

print(csr.subject)

builder = x509.CertificateBuilder()
builder = builder.subject_name(csr.subject)
builder = builder.issuer_name(ca.subject)
#builder = builder.not_valid_before(datetime.datetime.now())
builder = builder.not_valid_before(datetime.datetime.now()-datetime.timedelta(hours=1))
builder = builder.not_valid_after(datetime.datetime.now()+datetime.timedelta(7)) # days
builder = builder.public_key(csr.public_key())
builder = builder.serial_number(int(uuid.uuid4()))

for ext in csr.extensions:
    builder = builder.add_extension(ext.value, ext.critical)
    
certificate = builder.sign(private_key=ca_key, algorithm=hashes.SHA256(),backend=default_backend())

print(certificate.serial_number)

with open("openssl/client2.crt","wb") as f:
    f.write(certificate.public_bytes(serialization.Encoding.PEM))

<Name(CN=client 2)>
32196063756276690598583279125539955178


### 5. Client 3

In [92]:
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())

builder = x509.CertificateSigningRequestBuilder()
builder = builder.subject_name(x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u"client 3")
]))

builder = builder.add_extension(x509.BasicConstraints(ca=False, path_length=None), critical=True,)

request = builder.sign(private_key, hashes.SHA256(), default_backend())

with open("openssl/client3.csr","wb") as f:
    f.write(request.public_bytes(Encoding.PEM))

with open("openssl/client3.key","wb") as f:
    f.write(private_key.private_bytes(Encoding.PEM, PrivateFormat.TraditionalOpenSSL, NoEncryption()))

pem_csr = open("openssl/client3.csr","rb").read()
csr = x509.load_pem_x509_csr(pem_csr, default_backend())

pem_cert = open("openssl/ca.crt","rb").read()
ca = x509.load_pem_x509_certificate(pem_cert, default_backend())
pem_key = open("openssl/ca.key","rb").read()
ca_key = serialization.load_pem_private_key(pem_key, password=b"test", backend=default_backend())

print(csr.subject)

builder = x509.CertificateBuilder()
builder = builder.subject_name(csr.subject)
builder = builder.issuer_name(ca.subject)
#builder = builder.not_valid_before(datetime.datetime.now())
builder = builder.not_valid_before(datetime.datetime.now()-datetime.timedelta(hours=1))
builder = builder.not_valid_after(datetime.datetime.now()+datetime.timedelta(7)) # days
builder = builder.public_key(csr.public_key())
builder = builder.serial_number(int(uuid.uuid4()))

for ext in csr.extensions:
    builder = builder.add_extension(ext.value, ext.critical)
    
certificate = builder.sign(private_key=ca_key, algorithm=hashes.SHA256(),backend=default_backend())

print(certificate.serial_number)

with open("openssl/client3.crt","wb") as f:
    f.write(certificate.public_bytes(serialization.Encoding.PEM))

<Name(CN=client 3)>
315636500865185387148620898102571465908


###  6. Revocation list. Revoke a certificate

#### OpenSSL command
openssl ca -revoke client2.crt -cert ca.crt -keyfile ca.key

openssl ca -gencrl -out ca.crl -cert ca.crt -keyfile ca.key

In [97]:

# cert you want to revoke
cert_to_revoke_data = open("openssl/client1.crt","rb").read()
cert_to_revoke = x509.load_pem_x509_certificate(cert_to_revoke_data, backend=default_backend())

pem_cert = open("openssl/ca.crt","rb").read()
ca_crt = x509.load_pem_x509_certificate(pem_cert, default_backend())
pem_key = open("openssl/ca.key","rb").read()
ca_key = serialization.load_pem_private_key(pem_key, password=b"test", backend=default_backend())

# load crl
pem_crl_data = open("openssl/ca.crl","rb").read()
crl = x509.load_pem_x509_crl(pem_crl_data, backend=default_backend())

# generate a new crl object
builder = x509.CertificateRevocationListBuilder()
builder = builder.issuer_name(crl.issuer)
builder = builder.last_update(crl.last_update)
builder = builder.next_update(datetime.datetime.now() + datetime.timedelta(1, 0, 0))

# add crl certificates from file to the new crl object
for i in range(0,len(crl)):    
    builder = builder.add_revoked_certificate(crl[i])

# see if the cert to be revokek already in the list    
ret = crl.get_revoked_certificate_by_serial_number(cert_to_revoke.serial_number)

# if not, then add new revoked cert
if  not isinstance(ret, x509.RevokedCertificate):
    
    revoked_cert = x509.RevokedCertificateBuilder()\
    .serial_number(cert_to_revoke.serial_number)\
    .revocation_date(datetime.datetime.now()).build(backend=default_backend())
    
    builder = builder.add_revoked_certificate(revoked_cert)

# sign and save to new crl file
cert_revocation_list = builder.sign(private_key=ca_key,algorithm=hashes.SHA256(),backend=default_backend())

with open("openssl/ca.crl","wb") as f:
    f.write(cert_revocation_list.public_bytes(serialization.Encoding.PEM))
    
    

### Check certificates info

In [27]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID

pem_cert = open("openssl/ca.crt","rb").read()
cert = x509.load_pem_x509_certificate(pem_cert, default_backend())

print(cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(cert.not_valid_before)
print(cert.not_valid_after)

client_cert_data = open("openssl/client2.crt","rb").read()
client_cert = x509.load_pem_x509_certificate(client_cert_data, backend=default_backend())

print(client_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client_cert.not_valid_before)
print(client_cert.not_valid_after)

client_cert_data = open("openssl/client1.crt","rb").read()
client_cert = x509.load_pem_x509_certificate(client_cert_data, backend=default_backend())

print(client_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(client_cert.not_valid_before)
print(client_cert.not_valid_after)

Test Root CA
2020-11-08 17:50:18
2020-11-15 18:50:18
client 2
2020-11-08 17:48:26
2020-11-15 18:48:26
client 1
2020-11-08 17:48:24
2020-11-15 18:48:24
